In [255]:
using ToeplitzMatrices
using Base.Test
using IterativeLinearSolvers
import Base: full, getindex, print_matrix, size, tril, triu, *, inv, A_mul_B!, Ac_mul_B, A_ldiv_B!
import Base.\
import Base.LinAlg: BlasFloat, BlasReal, DimensionMismatch
export Toeplitz, SymmetricToeplitz, Circulant, Trian

In [4]:
ns=10 # small
nl = 2000 # large
xs = randn(ns)
xl= randn(nl)

As = SymmetricToeplitz(0.9.^( 0:ns-1 ))
Al = SymmetricToeplitz(0.9.^( 0:nl-1 ))
;

In [5]:
@test_approx_eq As*xs full(As)*xs
@test_approx_eq Al*xl full(Al)*xl
@test_approx_eq A_ldiv_B!(As,copy(xs)) full(As)\xs
@test_approx_eq A_ldiv_B!(Al,copy(xl)) full(Al)\xl

In [199]:
function durbin!{T<:BlasFloat}(r::AbstractVector{T}, y::AbstractVector{T})
    n = length(r)
    if n != length(y) throw(DimensionMismatch("Vector must have same length")) end
    y[1] = -r[1]
    β = one(T)
    α = -r[1]
    @inbounds for k = 1:n-1
        β *= one(T) - α*α
        α = -r[k+1]
        @simd for j = 1:k
            α -= r[k-j+1]*y[j]
        end
        α /= β
        @simd for j = 1:k÷2
            tmp = y[j]
            y[j] += α*y[k-j+1]
            y[k-j+1] += α*tmp
        end
        if isodd(k) 
            y[div(k,2)+1] *= one(T) + α
        end
        y[k+1] = α
    end
    return y
end
durbin(r::AbstractVector) = durbin!(r, zeros(length(r)))

durbin (generic function with 1 method)

In [239]:
@time y=durbin(Al.vc[2:end]);

  0.003670 seconds (8 allocations: 31.609 KB)


In [259]:
function trench!{T<:BlasFloat}(r::AbstractVector{T}, y::AbstractVector{T}, B::AbstractMatrix{T})
    n = length(r)+1
    if length(r) != length(y) throw(DimensionMismatch("Vector must have same length")) end
    if (n,n) != size(B) throw(DimensionMismatch("B must have same size (n,n)")) end
    γ::T = one(T)/(one(T)+dot(r,y))
    vrev = γ .* y
    v = reverse(vrev)    
    B[1,1] = B[end,end] = γ
    B[1,2:end] = vrev
    B[1:end-1,end] = v
    @inbounds for i in 2:(((n-1)÷2)+1)
        for j in i:(n-i+1)
             B[i,j] = B[i-1,j-1]+(v[n+1-j]*v[n+1-i]-v[i-1]*v[j-1])/γ
             B[end-j+1,end-i+1] = B[i,j]
        end
    end
    return B
end
function trench(r::AbstractVector, y::AbstractVector)
    n = length(r)+1
    B = ones(Float64, (n,n))
    return trench!(r, y, B)
end
function inv{T<:BlasFloat}(A::SymmetricToeplitz{T})
    r = A.vc[2:end]./A.vc[1]
    y=durbin(r)
    return Symmetric(trench(r,y))./A.vc[1]
end

inv (generic function with 31 methods)

In [231]:
@time Symmetric(trench(Al.vc[2:end], y));

  0.095140 seconds (25.16 k allocations: 31.678 MB, 46.06% gc time)


In [270]:
r = Al.vc[2:end]
B = ones(Float64, nl, nl)
@code_warntype trench!(r, y, B);

Variables:
  r::Array{Float64,1}
  y::Array{Float64,1}
  B::Array{Float64,2}
  n::Int64
  γ::Float64
  vrev::Array{Float64,1}
  v::Array{Float64,1}
  #s2420::Int64
  i::Int64
  #s2421::Int64
  j::Int64
  ##I#13538::Tuple{}
  ##I#13539::Tuple{}
  ##I#13540::Tuple{Int64,UnitRange{Int64}}
  ####J#11450#13541::Tuple{Int64,UnitRange{Int64}}
  ######I#11438#11451#13542::Tuple{Int64,UnitRange{Int64}}
  #########s21#11419#11439#11452#13543::Bool
  ########_var0#11420#11440#11453#13544::Bool
  ###########s20#11414#11421#11441#11454#13545::Bool
  ##########_var0#11415#11422#11442#11455#13546::Int64
  ##########_var1#11416#11423#11443#11456#13547::Bool
  ##########_var2#11417#11424#11444#11457#13548::Int64
  ##########_var3#11418#11425#11445#11458#13549::Bool
  ########_var4#11426#11446#11459#13550::Bool
  ######_var1#11447#11460#13551::Bool
  ######J#11448#11461#13552::Tuple{Int64,UnitRange{Int64}}
  ########I#11437#11449#11462#13553::Tuple{UnitRange{Int64}}
  ##I#13554::Tuple{UnitRange{Int64},I

In [262]:
@time inv(Al)

  0.056081 seconds (22 allocations: 61.112 MB)


2000x2000 Array{Float64,2}:
  5.26316       -4.73684        7.85747e-31   …   6.81274e-106  -1.69847e-106
 -4.73684        9.52632       -4.73684          -1.80585e-105   6.81274e-106
  7.85747e-31   -4.73684        9.52632           1.92644e-105  -1.1927e-105 
  8.95356e-31   -2.00734e-32   -4.73684          -2.5628e-106    8.53007e-106
 -8.88541e-31    1.69504e-30   -2.00734e-32      -1.65487e-105   5.11427e-106
 -5.81404e-32   -8.36215e-31    1.69504e-30   …   1.92436e-105  -1.19459e-105
  7.67891e-31   -7.49242e-31   -8.36215e-31      -1.78716e-105   8.49233e-106
 -8.71125e-31    1.5519e-30    -7.49242e-31       2.28312e-105  -1.02285e-105
  6.89788e-31   -1.49193e-30    1.5519e-30       -2.58884e-105   1.36255e-105
 -4.45794e-31    1.091e-30     -1.49193e-30       2.58884e-105  -1.36255e-105
  2.04385e-31   -6.29741e-31    1.091e-30     …  -2.24915e-105   1.36255e-105
 -6.53706e-32    2.63219e-31   -6.29741e-31       1.26026e-105  -1.02285e-105
  1.38393e-15   -1.24554e-15    2.63

In [83]:
@time inv(full(Al))

  0.460745 seconds (22 allocations: 92.545 MB, 8.26% gc time)


2000x2000 Array{Float64,2}:
  5.26316       -4.73684       -5.84328e-18   …   5.58059e-106  -1.05494e-106
 -4.73684        9.52632       -4.73684          -1.41863e-105   4.79911e-106
  0.0           -4.73684        9.52632           1.52895e-105  -1.012e-105  
  6.21725e-16    1.77636e-15   -4.73684          -1.54806e-106   7.07128e-106
 -1.68754e-15    1.33227e-15    4.44089e-15      -1.66799e-105   6.85048e-106
 -6.21725e-16   -5.32907e-16   -2.84217e-15   …   2.40464e-105  -1.524e-105  
  2.84217e-15   -5.4623e-15     6.30607e-15      -2.4547e-105    1.2546e-105 
 -8.43769e-16    3.64153e-15   -6.75016e-15       1.82204e-105  -7.68461e-106
 -1.33227e-16   -1.33227e-16    1.86517e-15      -8.76465e-106   4.22163e-106
  8.15137e-16   -3.92668e-16   -1.0664e-15        1.27274e-105  -7.01844e-106
 -7.90011e-16    3.09694e-17    1.99197e-15   …  -1.85355e-105   1.24403e-105
 -1.02024e-15    2.34841e-15   -1.49062e-15       2.04118e-105  -1.5022e-105 
  2.40393e-15   -4.30358e-15    1.41

In [37]:
any(Symmetric(B).==1.0)

false

In [263]:
@test_approx_eq(inv(Al)*full(Al),eye(nl))

In [264]:
@test_approx_eq(inv(As)*full(As),eye(ns))

In [265]:
Ab = SymmetricToeplitz(0.9.^( 0:ns-1 ))
Ab.vc[1] = 2.0
@test_approx_eq(inv(Ab)*full(Ab),eye(ns))